In [ ]:
#====================================================================================================#
#      Competición - SPAM/NOT SPAM - ITERACIÓN 8 (FINAL)                                            #
#      V8: V3 PURO + MICRO-AJUSTES                                                                  #
#                                                                                                    #
#      LECCIONES APRENDIDAS:                                                                        #
#      - V3 (LSTM puro): 0.8849 val / 0.87 test ← MEJOR                                            #
#      - V4 (DistilBERT): 0.64 ← FRACASO                                                           #
#      - V5 (CNN+LSTM): 0.81-0.86 ← overfitting                                                    #
#      - V6 (attention): 0.78-0.82 ← overfitting                                                   #
#      - V7 (features): 0.8610 ← peor que V3                                                       #
#                                                                                                    #
#      ESTRATEGIA V8:                                                                               #
#      ✅ V3 EXACTO (lo que funciona)                                                               #
#      🔧 LSTM: 64 → 72 (+12.5% capacidad)                                                          #
#      🔧 L2: 5e-4 → 4e-4 (ligeramente menos restrictivo)                                          #
#      🔧 Patience: 2 → 3 (dar más tiempo a converger)                                             #
#                                                                                                    #
#      OBJETIVO: MCC >= 0.89                                                                        #
#====================================================================================================#

In [ ]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

seed = 42
np.random.seed(seed)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, LSTM, Bidirectional, Dense, Dropout,
    GlobalMaxPooling1D, SpatialDropout1D
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

tf.random.set_seed(seed)
keras.utils.set_random_seed(seed)

from sklearn.metrics import matthews_corrcoef, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
# Hyperparameters - V8 = V3 + micro-ajustes
MAX_WORDS = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100

# Model - MICRO-AJUSTES vs V3
LSTM_UNITS = 72        # V3: 64 → 72 (+12.5% capacidad)
DENSE_UNITS = 32       # IGUAL V3
SPATIAL_DROPOUT = 0.4  # IGUAL V3
DROPOUT_RATE = 0.7     # IGUAL V3
L2_REG = 4e-4          # V3: 5e-4 → 4e-4 (menos restrictivo)

# Training
BATCH_SIZE = 32
EPOCHS = 50
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 5e-4   # IGUAL V3
CLIPNORM = 1.0         # IGUAL V3

print("V8 Config: LSTM=72, L2=4e-4, patience=3")
print("Cambios vs V3: +12.5% capacidad LSTM, -20% L2 regularización")

In [ ]:
# Load data
train = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/train.csv", index_col="row_id")
test = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/test.csv", index_col="row_id")

print(f"Train: {len(train)} | Test: {len(test)}")

In [ ]:
# Tokenization - IGUAL V3
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='<OOV>')
tokenizer.fit_on_texts(train['text'])

X_train_seq = tokenizer.texts_to_sequences(train['text'])
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')
y_train = train['spam_label'].values

X_test_seq = tokenizer.texts_to_sequences(test['text'])
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Train/val split
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train_pad, y_train, test_size=VALIDATION_SPLIT, random_state=seed, stratify=y_train
)

print(f"Train: {len(X_train_final)} | Val: {len(X_val)}")

In [ ]:
# Model V8 - V3 PURO con micro-ajustes
model = Sequential([
    Embedding(
        input_dim=MAX_WORDS,
        output_dim=EMBEDDING_DIM,
        input_length=MAX_LEN
    ),
    SpatialDropout1D(SPATIAL_DROPOUT),
    Bidirectional(
        LSTM(
            LSTM_UNITS,
            return_sequences=True,
            kernel_regularizer=l2(L2_REG),
            recurrent_regularizer=l2(L2_REG),
            bias_regularizer=l2(L2_REG)
        )
    ),
    GlobalMaxPooling1D(),
    Dense(
        DENSE_UNITS,
        activation='relu',
        kernel_regularizer=l2(L2_REG),
        bias_regularizer=l2(L2_REG)
    ),
    Dropout(DROPOUT_RATE),
    Dense(1, activation='sigmoid')
], name='V8_LSTM_Pure_Tuned')

optimizer = keras.optimizers.AdamW(
    learning_rate=LEARNING_RATE,
    weight_decay=1e-4,
    clipnorm=CLIPNORM
)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()]
)

model.summary()

In [ ]:
# Callbacks - ligeramente más paciencia
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=3,  # V3: 2 → 3 (más tiempo)
        restore_best_weights=True,
        verbose=1
    ),
    ModelCheckpoint(
        'best_spam_model_v8.keras',
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,  # V3: 1 → 2
        min_lr=1e-6,
        verbose=1
    )
]

history = model.fit(
    X_train_final, y_train_final,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# Evaluation
y_pred_proba = model.predict(X_val, batch_size=BATCH_SIZE, verbose=0).flatten()

best_threshold = 0.5
y_pred = (y_pred_proba > best_threshold).astype(int)
mcc_val = matthews_corrcoef(y_val, y_pred)

# Debug
final_epoch = len(history.history['loss'])
train_loss_final = history.history['loss'][-1]
val_loss_final = history.history['val_loss'][-1]
overfitting_delta = val_loss_final - train_loss_final

spam_probs = y_pred_proba[y_val == 1]
notspam_probs = y_pred_proba[y_val == 0]
separation = spam_probs.mean() - notspam_probs.mean()

print("="*80)
print("V8 - LSTM PURE + MICRO-TUNING RESULTS")
print("="*80)
print(f"Val MCC: {mcc_val:.4f}")
print(f"Epochs: {final_epoch}")
print(f"Train Loss: {train_loss_final:.4f} | Val Loss: {val_loss_final:.4f}")
print(f"Overfitting Δ: {overfitting_delta:.4f}")
print(f"Class Separation: {separation:.4f}")
print("="*80)
print(classification_report(y_val, y_pred, target_names=['Not SPAM', 'SPAM']))
print("="*80)
print(f"\nCOMPARACIÓN:")
print(f"V3 (baseline):     0.8849 val / 0.87 test")
print(f"V7 (features):     0.8610 val ← PEOR")
print(f"V8 (micro-tuned):  {mcc_val:.4f} val")

if mcc_val > 0.8849:
    print("\n✅ V8 SUPERA V3 - USAR ESTE MODELO")
elif mcc_val > 0.88:
    print("\n✓ V8 similar a V3 - Probar en test")
else:
    print("\n⚠️ V8 no supera V3 - V3 sigue siendo el mejor")

In [ ]:
# Test predictions
y_test_proba = model.predict(X_test_pad, batch_size=BATCH_SIZE, verbose=0).flatten()
y_test_pred = (y_test_proba > best_threshold).astype(int)

submission = pd.read_csv("/kaggle/input/u-tad-spam-not-spam-2025-edition/sample_submission.csv")
submission["spam_label"] = y_test_pred
submission.to_csv('submission.csv', index=False)

print(f"\nSubmission: {len(y_test_pred)} predictions")
print(f"SPAM: {y_test_pred.sum()} ({y_test_pred.mean()*100:.1f}%)")
print(f"Not SPAM: {len(y_test_pred) - y_test_pred.sum()} ({(1-y_test_pred.mean())*100:.1f}%)")